## Run job

In [ ]:
# This scripts starts a job on a cluster.
# Run this script to run the source code in ./src/main.py
#
from azure.ai.ml import MLClient, command, Input, Output
from azure.identity import DefaultAzureCredential

# Create the ML Client
ml_client = MLClient.from_config(DefaultAzureCredential())


### Run task creation

In [ ]:
# Prepare input directories
inp_uri = f'azureml://subscriptions/f8c5aac3-29fc-4387-858a-1f61722fb57a/resourcegroups/forskerpl-n0ybkr-rg/workspaces/forskerpl-n0ybkr-mlw/datastores/workspaceblobstore/paths/MEDS/data/MEDS_data/'
inputs = {"input_dir": Input(path=inp_uri)}

# Prepare output directories
out_uri = f'azureml://subscriptions/f8c5aac3-29fc-4387-858a-1f61722fb57a/resourcegroups/forskerpl-n0ybkr-rg/workspaces/forskerpl-n0ybkr-mlw/datastores/workspaceblobstore/paths/MEDS/data/MEDS_data/'
outputs = {"output_dir":Output(path=inp_uri)}

In [ ]:
# Chose a compute cluster target (review the options at "Compute" => "Compute clusters")
compute_target = "CPU-20-LP"

# Start the job
job = command(
    code=".", # Folder with source code
    command="bash task_example.sh ${{outputs.output_dir}} CPH_data cancer/all_cancers ",
    inputs=inputs,
    outputs=outputs,
    environment="meds_dev@latest",
    compute=compute_target,
    #name="zsxdcfgh"
)

returned_job = ml_client.create_or_update(job, experiment_name="MEDS_tasks")

### Run tabs

In [ ]:
# Prepare input directories
inp_uri = f'azureml://subscriptions/f8c5aac3-29fc-4387-858a-1f61722fb57a/resourcegroups/forskerpl-n0ybkr-rg/workspaces/forskerpl-n0ybkr-mlw/datastores/workspaceblobstore/paths/MEDS/data/MEDS_data/'
inp_uri_labels = f'azureml://subscriptions/f8c5aac3-29fc-4387-858a-1f61722fb57a/resourcegroups/forskerpl-n0ybkr-rg/workspaces/forskerpl-n0ybkr-mlw/datastores/workspaceblobstore/paths/MEDS/data/MEDS_data/task_labels'
inputs = {"input_dir": Input(path=inp_uri), "labels_dir": Input(path=inp_uri_labels)}

# Prepare output directories
out_uri = f'azureml://subscriptions/f8c5aac3-29fc-4387-858a-1f61722fb57a/resourcegroups/forskerpl-n0ybkr-rg/workspaces/forskerpl-n0ybkr-mlw/datastores/workspaceblobstore/paths/MEDS/data/MEDS_data_reshard/'
out_uri_tabs = f'azureml://subscriptions/f8c5aac3-29fc-4387-858a-1f61722fb57a/resourcegroups/forskerpl-n0ybkr-rg/workspaces/forskerpl-n0ybkr-mlw/datastores/workspaceblobstore/paths/MEDS/data/MEDS_data_tabs_2/'
outputs = {"output_reshard_dir":Output(path=out_uri), "output_dir":Output(path=out_uri_tabs)}

In [ ]:
# Chose a compute cluster target (review the options at "Compute" => "Compute clusters")
compute_target = "CPU-20-LP"

# Start the job
job = command(
    code=".", # Folder with source code
    command="bash -i run_tabs.sh ${{inputs.input_dir}}  ${{outputs.output_reshard_dir}} ${{outputs.output_dir}} 'cancer/all_cancers' ${{inputs.labels_dir}} ../models/ 200 ",
    inputs=inputs,
    outputs=outputs,
    environment="meds_tabs@latest",
    compute=compute_target,
    mode="download",
    #name="zsxdcfgh"
)

returned_job = ml_client.create_or_update(job, experiment_name="MEDS_tasks")

## Make env

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Environment

mlc = MLClient.from_config(DefaultAzureCredential())

env = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="../envs/tab_environment_simple.yaml",
    name="meds_tabs",
    description="Environment for MEDS-tabs",
)
mlc.environments.create_or_update(env) 